In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy

In [2]:
# Дано:

P_el = 100*10**3 # Требуемая электрическая мощность реактора
Eta = 0.0348227272727273 # КПД реактора

Q_izl_treb = 3326025.3230648716 # Требуемое количество излучаемого тепла с радиатора
eps = 0.88 # Степень черноты материала радиатора
sig = 5.67*10**-8 # Постоянная Стефана-Больцмана
T = 800 # К - Температура радиатора
delta = 0.009425235965461689 # Толщина радиатора
ro_izl = 8920 # кг/м3 - Плотность материала радиатора (медь)
G_izl = 13682.232265432767 # кг - Масса радиатора

A = 1.07 # Коэффициент запаса

k = 0.003 # Коэффициент вариации
n = 40000 # Заданное количество реализаций случайной величины
gamma = 0.997

In [3]:
# Генерация случайных величин распределенных по нормальному закону
P_el_n = np.random.normal(P_el, k*P_el, n)
Eta_n = np.random.normal(Eta, k*Eta, n)

ro_izl_n = np.random.normal(ro_izl, k*ro_izl, n)
# sig_n = np.random.normal(sig, k*sig, n)
T_n = np.random.normal(T, k*T, n)
delta_n = np.random.normal(delta, k*delta, n)
G_izl_n = np.random.normal(G_izl, k*G_izl, n)
eps_n = np.random.normal(eps, k*eps, n)

# Q_izl_treb_n = np.random.normal(Q_izl_treb, k*Q_izl_treb, n)

In [4]:
# Генерация случайных величин распределенных по закону равномерной плотности

A_P_el = P_el-np.sqrt(3)*P_el*k
B_P_el = P_el+np.sqrt(3)*P_el*k
P_el_un=np.random.uniform(A_P_el, B_P_el, n)

A_Eta = Eta-np.sqrt(3)*Eta*k
B_Eta = Eta+np.sqrt(3)*Eta*k
Eta_un=np.random.uniform(A_Eta, B_Eta, n)

A_ro_izl = ro_izl-np.sqrt(3)*ro_izl*k
B_ro_izl = ro_izl+np.sqrt(3)*ro_izl*k
ro_izl_un=np.random.uniform(A_ro_izl, B_ro_izl, n)

A_T = T-np.sqrt(3)*T*k
B_T = T+np.sqrt(3)*T*k
T_un=np.random.uniform(A_T, B_T, n)

A_delta = delta-np.sqrt(3)*delta*k
B_delta = delta+np.sqrt(3)*delta*k
delta_un=np.random.uniform(A_delta, B_delta, n)

A_G_izl = G_izl-np.sqrt(3)*G_izl*k
B_G_izl = G_izl+np.sqrt(3)*G_izl*k
G_izl_un=np.random.uniform(A_G_izl, B_G_izl, n)

A_eps = eps-np.sqrt(3)*eps*k
B_eps = eps+np.sqrt(3)*eps*k
eps_un=np.random.uniform(A_eps, B_eps, n)

In [5]:
# Q_izl_treb = (G_izl/(delta*ro_izl))*eps*sig*T**4
Q_izl_treb_n = (1.2 * P_el_n) / Eta_n
Q_izl_treb_un = (1.2 * P_el_un) / Eta_un

Q_izl_i_norm = (G_izl_n/(delta_n*ro_izl_n))*A*eps_n*sig*T_n**4
Q_izl_i_un = (G_izl_un/(delta_un*ro_izl_un))*A*eps_un*sig*T_un**4

P_n = (Q_izl_i_norm>=Q_izl_treb_n).mean()
tgamma_n = stats.norm.ppf((1+gamma)/2,0,1)
delta1_n = tgamma_n*np.sqrt((1-P_n)/(P_n*n))
P1_n = P_n+tgamma_n*np.sqrt(P_n*(1-P_n)/n)
P2_n = P_n-tgamma_n*np.sqrt(P_n*(1-P_n)/n)

P_un = (Q_izl_i_un>=Q_izl_treb_un).mean()
tgamma_un = stats.uniform.ppf((1+gamma)/2,0,1)
delta1_un = tgamma_un*np.sqrt((1-P_un)/(P_un*n))
P1_un = P_un+tgamma_un*np.sqrt(P_un*(1-P_un)/n)
P2_un = P_un-tgamma_un*np.sqrt(P_un*(1-P_un)/n)

In [6]:
print('Результаты метода статистических испытаний для нормального закона распределения и закона равномерной плотности:'
     f'\nКоличество испытаний для каждого закона =', n,
     f'\nP_норм =', P_n,
     f'\n\nВерхняя граница доверительного интервала при нормальном законе распределения =', P1_n,
     f'\nНижняя граница доверительного интервала при нормальном законе распределения =', P2_n,
     f'\nПогрешность определения ВБР при нормальном законе распределения =', delta1_n,
     f'\n\nP_р.п =', P_un,
     f'\n\nВерхняя граница доверительного интервала при з.р.п =', P1_un,
     f'\nНижняя граница доверительного интервала при з.р.п =', P2_un,
     f'\nПогрешность определения ВБР при з.р.п =', delta1_un)

Результаты метода статистических испытаний для нормального закона распределения и закона равномерной плотности:
Количество испытаний для каждого закона = 40000 
P_норм = 0.988725 

Верхняя граница доверительного интервала при нормальном законе распределения = 0.9902917207090979 
Нижняя граница доверительного интервала при нормальном законе распределения = 0.987158279290902 
Погрешность определения ВБР при нормальном законе распределения = 0.0015845869266964405 

P_р.п = 0.995475 

Верхняя граница доверительного интервала при з.р.п = 0.9958100753996139 
Нижняя граница доверительного интервала при з.р.п = 0.9951399246003861 
Погрешность определения ВБР при з.р.п = 0.00033659850786191
